In [13]:
import torch.nn as nn
import torch
import MST
import numpy as np

In [14]:
np.random.seed(42)

In [15]:
in_C = 1
out_C = 1
stride = 1
padding = 1
dilation = 1
kernel = 3

params = [in_C, out_C, kernel, stride, padding, dilation]

print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")

16.0


In [16]:
with torch.no_grad():
    myConv = MST.Conv2d(*params)
    torchConv = nn.Conv2d(*params)

    torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
    torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())


In [17]:
image = np.ones((1, in_C, 16, 16))

In [18]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[:20])
    print(nnres[:20])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

(1, 1, 16, 16)
torch.Size([1, 1, 16, 16])
[-0.08   1.382  1.382  1.382  1.382  1.382  1.382  1.382  1.382  1.382
  1.382  1.382  1.382  1.382  1.382  1.714  0.16   1.856  1.856  1.856]
[-0.08   1.382  1.382  1.382  1.382  1.382  1.382  1.382  1.382  1.382
  1.382  1.382  1.382  1.382  1.382  1.714  0.16   1.856  1.856  1.856]
0


In [19]:
%%timeit
myres = myConv(image)

123 µs ± 2.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
%%timeit
nnres = torchConv(torch.Tensor(image))

44.5 µs ± 2.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
def to_np_arr(a):
    a_ = MST.MDT_ARRAY(a).astype(np.float32) / 255
    return a_.reshape(1, *a_.shape)

import torch.nn as nn
import torch
import MST
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets
np.random.seed(42)
in_C = 1
out_C = 23
stride = 2
padding = 1
dilation = 1
kernel = 3
inH, InW = 32, 32
outH, outW = inH//stride, InW//stride

params = [in_C, out_C, kernel, stride, padding, dilation]

print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")
myConv = MST.Conv2d(*params)
torchConv = nn.Conv2d(*params)

torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())
image = np.ones((1, in_C, inH, InW))

transform = to_np_arr

8.5


In [25]:
# torchImage = torch.ones((1, in_C, inH, InW), requires_grad=True)

from tqdm.auto import tqdm
train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=0,
)

for (images, labels) in tqdm(train_dataloader):

    torchImage = images
    image = torchImage.detach().numpy()
    
    out_sample = myConv(image)

    dOut = np.random.random(out_sample.shape)

    myres = myConv(image)
    myres.backward(dOut)
    nnres = torchConv(torchImage)
    nnres.backward(torch.Tensor(dOut))

    # nngrad = torchImage.grad.numpy().round(3).flatten()
    # mygrad = myConv._dinX.round(3).flatten()
    # bads = np.where(abs(mygrad - nngrad) > 0.001)
    # print(len(bads[0]))
    # for pos in bads[0][:10]:
    #     print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    nnres = nnres.detach().numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

    nngrad = torchConv.weight.grad.numpy().round(3).flatten()
    mygrad = myConv._dw.round(3).flatten()
    bads = np.where(abs(mygrad - nngrad) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    torchConv.weight.grad = torch.zeros_like(torchConv.weight.grad)

    nngrad = torchConv.bias.grad.numpy().round(3).flatten()
    mygrad = myConv._dbias.round(3).flatten()
    bads = np.where(abs(mygrad - nngrad) > 0.001)
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

    torchConv.bias.grad = torch.zeros_like(torchConv.bias.grad)
    break

  0%|          | 0/1875 [00:00<?, ?it/s]

0
207
[0]: 386.081 396.502
[1]: 385.378 398.615
[2]: 374.968 392.748
[3]: 388.085 389.532
[4]: 387.335 394.081
[5]: 377.005 388.213
[6]: 382.566 386.309
[7]: 382.936 393.387
[8]: 374.099 386.627
[9]: 374.092 386.094
12
[0]: 3162.837 3162.836
[1]: 3096.410 3096.409
[3]: 3137.900 3137.896
[8]: 3170.087 3170.089
[10]: 3137.948 3137.951
[15]: 3134.526 3134.527
[16]: 3126.596 3126.595
[17]: 3109.952 3109.951
[18]: 3166.328 3166.325
[19]: 3116.705 3116.703


In [26]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[:20])
    print(nnres[:20])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

(32, 23, 14, 14)
torch.Size([32, 23, 14, 14])
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0


In [9]:
import torch.nn as nn
import torch
import MST
import numpy as np
np.random.seed(42)
BS = 5
in_C = 2
out_C = 128
stride = 1
padding = 0
dilation =1
kernel = 3
inH, InW = 32, 32
outH, outW = inH//stride, InW//stride

params = [in_C, out_C, kernel, stride, padding, dilation]

print(f"{(16 - kernel + padding*2 - (kernel-1)*(dilation-1))/stride + 1}")
myConv = MST.Conv2d(*params)
torchConv = nn.Conv2d(*params)

torchConv.weight = nn.Parameter(torch.tensor(myConv._w))
torchConv.bias = nn.Parameter(torch.tensor(myConv._bias).flatten())
image = np.ones((BS, in_C, inH, InW))

14.0


In [10]:
image = np.random.random((BS, in_C, inH, InW))
aboba = MST.col2im_2(MST.im2col_3(image, (inH, InW), (1, 1), (1,1)), image.shape, (inH, InW), (1,1), (1,1), (0,0))
bads = np.where(abs(image - aboba) > 0.001)
bads

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))

In [11]:
with torch.no_grad():
    myres = myConv(image)
    nnres = torchConv(torch.Tensor(image))
    print(myres.shape)
    print(nnres.shape)

    nnres = nnres.numpy().round(3).flatten()
    myres = myres.round(3).flatten()
    bads = np.where(abs(nnres - myres) > 0.001)
    print(myres[54:70])
    print(nnres[54:70])
    print(len(bads[0]))
    for pos in bads[0][:10]:
        print(f"[{pos}]: {myres[pos]:.3f} {nnres[pos]:.3f}")

(5, 128, 30, 30)
torch.Size([5, 128, 30, 30])
[-0.562  0.255 -0.362  0.057  0.006 -0.277 -0.769 -0.223  0.031 -0.475
 -0.372 -0.028 -0.004 -0.54  -0.508  0.141]
[-0.562  0.255 -0.362  0.057  0.006 -0.277 -0.769 -0.223  0.031 -0.475
 -0.372 -0.028 -0.004 -0.54  -0.508  0.141]
16
[3395]: 0.278 0.279
[25392]: -0.809 -0.810
[79893]: -0.937 -0.936
[104308]: -0.221 -0.220
[130645]: 1.449 1.448
[152817]: -0.536 -0.537
[157400]: -0.099 -0.100
[168547]: -0.701 -0.702
[305819]: 0.807 0.806
[325181]: 0.370 0.369


In [12]:
torchImage = torch.ones((BS, in_C, inH, InW), requires_grad=True)

out_sample = myConv(image)

myConv(image).backward(np.ones(out_sample.shape))
torchConv(torchImage).backward(torch.ones(out_sample.shape))


nngrad = torchImage.grad.numpy().round(3).flatten()
mygrad = myConv._dinX.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")


nngrad = torchConv.weight.grad.numpy().round(3).flatten()
mygrad = myConv._dw.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

torchConv.weight.grad = torch.zeros_like(torchConv.weight.grad)

nngrad = torchConv.bias.grad.numpy().round(3).flatten()
mygrad = myConv._dbias.round(3).flatten()
bads = np.where(abs(mygrad - nngrad) > 0.001)
print(len(bads[0]))
for pos in bads[0][:10]:
    print(f"[{pos}]: {mygrad[pos]:.3f} {nngrad[pos]:.3f}")

torchConv.bias.grad = torch.zeros_like(torchConv.bias.grad)

0
2304
[0]: 2233.720 4500.000
[1]: 2235.277 4500.000
[2]: 2241.223 4500.000
[3]: 2234.828 4500.000
[4]: 2234.824 4500.000
[5]: 2240.548 4500.000
[6]: 2222.335 4500.000
[7]: 2223.530 4500.000
[8]: 2230.652 4500.000
[9]: 2230.503 4500.000
0
